In [5]:
%pip install chess
%pip install opencv-python
%pip install tensorflow

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.



In [2]:
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import chess
import chess.svg

In [4]:
def preprocess_image(image_path):
    image = cv2.imread(image_path)

    if image is None:
        print(f"Error: Could not load image at {image_path}")
        return None
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    blurred = cv2.GaussianBlur(gray, (5, 5), 0)

    edges = cv2.Canny(blurred, 50, 150)

    return edges

image_path = './data/Chess ID Public Data/output_train/bb/0647_2.jpg'
edges = preprocess_image(image_path)

if edges is not None:
    cv2.imshow('Edges', edges)
    cv2.waitKey(0)
    cv2.destroyAllWindows()


In [6]:
def detect_chessboard(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    chessboard_size = (8, 8)
    ret, corners = cv2.findChessboardCorners(gray, chessboard_size)

    if ret:
        cv2.drawChessboardCorners(image, chessboard_size, corners, ret)
    
    return image, ret

image = cv2.imread(image_path)
chessboard_image, detected = detect_chessboard(image)

# Display the detected chessboard
if detected:
    cv2.imshow('Detected Chessboard', chessboard_image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()


In [8]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

def build_chess_piece_classifier(input_shape=(224, 224, 3), num_classes=12):
    base_model = tf.keras.applications.MobileNetV2(weights='imagenet', include_top=False, input_shape=input_shape)
    base_model.trainable = False

    model = tf.keras.Sequential([
        base_model,
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(num_classes, activation='softmax'), 
    ])

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

train_dir = './data/Chess ID Public Data/output_train'
val_dir = './data/Chess ID Public Data/output_test'

train_datagen = ImageDataGenerator(rescale=1./255, rotation_range=20, zoom_range=0.2)
val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(train_dir, target_size=(224, 224), batch_size=32, class_mode='categorical')
val_generator = val_datagen.flow_from_directory(val_dir, target_size=(224, 224), batch_size=32, class_mode='categorical')

model = build_chess_piece_classifier(num_classes=13)
model.fit(train_generator, validation_data=val_generator, epochs=1)

# Save the model for later use
model.save('chess_piece_classifier.keras')


Found 10360 images belonging to 13 classes.
Found 3646 images belonging to 13 classes.
324/324 ━━━━━━━━━━━━━━━━━━━━ 485s 1s/step - accuracy: 0.8166 - loss: 0.6027 - val_accuracy: 0.9984 - val_loss: 0.0082
